深度学习的可解释化工具Grad-CAM相信大家都不陌生啦~本文介绍该工具的使用，找了kaggle上较好的notebook翻译后分享给大家，该notebook使用keras实现。
原文地址：https://www.kaggle.com/ratthachat/aptos-augmentation-visualize-diabetic-retinopathy/notebook

因为是可视化工具，那肯定要有问题有模型，再使用这个工具来看模型的识别情况，辅助我们决策。
因为本文是kaggle竞赛里的一个kernel，所以大家需要了解一些背景情况：
* 竞赛题目：通过眼球图片来识别糖尿病视网膜病变的严重程度。分为5个严重等级：0，1，2，3，4。0代表正常眼球，4代表非常严重。
* 文中会出现一些hard exudates等专业名词，指的是医学上对眼球情况的描述。比如这里翻译为：坚硬的渗出物。还有wood spots，大家知道这是对眼球的描述就好了，请原谅我蹩脚的英语ヾ(≧O≦)〃嗷~（我是硬翻译的，有更好地翻译的同学请留言）。
* 关于Grad-CAM的论文。Grad-CAM大家可以去找找资料，这里主要讲keras的实现方式。最早使用CAM来可视化，CAM提取最后一层卷积池化层，然后将后面的层改为GAP层，重新训练权重。从而得到GAP层对于每个类别的权重。由于GAP层还没有丢失空间信息，所以我们可以依据权重来对特征图上采样。上采样的结果和原来的图片融合在一起，就是CAM了，感兴趣的同学可以看CAM的实现：https://github.com/philipperemy/tensorflow-class-activation-mapping
* Grad-CAM可以不需要再训练一遍，而且对没有GAP层的网络很包容。具体大家往下看哦~




# 1. 介绍 ：这些图片是非常严重的，模型有很好地预测到吗？

你想要了解，模型到底关注了图片哪些内容？和眼球图片致病区域是同一块地方吗？下面的图片显示了CNN真正关注了哪些区域，它是否真的理解了眼球致病的原因。

看下面的图片：
在第一个案例中，模型看起来运行不错。它可以指出眼球中的缺陷。
在第二个案例中，模型完全没有指出在眼球中间的wool spots，虽然它将图片评估为等级3.这个等级说明了我们模型还没有抓取hard exudates特征。
参考网址： https://www.eyeops.com/

![grad-cam](https://i.ibb.co/6FM6VCC/gradcam-resized.png)

![ref https://www.eyeops.com/](https://sa1s3optim.patientpop.com/assets/images/provider/photos/1947516.jpeg)

本次使用的可视化技术名字为“Grad-CAM”，你可以点击这些链接来找到该技术的原始材料： ([Gradient-weighted Class Activation Mapping](http://gradcam.cloudcv.org/) ;在这里，我会陈述怎么通过该技术得到更多关于你模型的见解。

我倾向于使用keras，所以我选择可视化他的模型，链接是：[public Keras model of @xhlulu](https://www.kaggle.com/xhlulu/aptos-2019-densenet-keras-starter)，他的模型在我写kernel的时候分数最高了。（你好棒 @xhlulu!）。对于喜欢pytorch的人，你可以看 [this kernel of @daisukelab](https://www.kaggle.com/daisukelab/verifying-cnn-models-with-cam-and-etc-fast-ai)，这篇文章也用到了这个技术。

keras版本引用了[this article](http://www.hackevolve.com/where-cnn-is-looking-grad-cam/)，该文章引用了F.Chollet的书

## 1.1 Grad-CAM简介

![](http://gradcam.cloudcv.org/static/images/network.png)

Grad-CAM可视化可以被直观地、非正式化地解释为：
**目标**着重强调模型做决策的像素区域（空间信息），使用热力图来可视化这些区域，如上图所示。
**简单描述**
* 我们相信最重要的空间信息来自于最后一层卷积层（在全局池化层前一层），也就是离全连接层最近的带有空间信息的层。
* 对于该层的每一通道，那些激活的图片区域。也就是说，我们希望该层每个通道可以抓取不同类别的特征。比如，通道0代表了class 0的激活区域，通道1代表了class 1 的激活区域。多分类问题还有多个通道。
* 我们通过计算结果和该层特征的梯度来强调图片中的重要信息。
* 因此，我们通过将计算出来的梯度和该层的tensor相乘来获取热力图。
* 最后我们将热力图的通道取平均，通过relu移除负值来获取最终热力图。
更多信息可以阅读作者的paper。


In [ ]:
import json
import math
import os

import cv2
from PIL import Image
import numpy as np
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm

%matplotlib inline

# 2. 准备原始kernel的工具

为了使用Grad-CAM来可视化模型，我们不需要再重新训练模型，我们只要直接用已经训练好的来自@xhlulu的原始kernel的模型权重。因此，在实际上，我们没有必要重新预处理训练/测试数据，我们会在线预处理。注意了，原始kernel使用了224x224的图片尺寸，我们使用图片预处理功能，如下面的代码所示。

尽管如此，为了保证我们加载了正确的模型权重，我会预处理测试数据，来确保我们和原始kernel一样预测正确。

In [ ]:
train_df = pd.read_csv('../input/aptos2019-blindness-detection/train.csv')
test_df = pd.read_csv('../input/aptos2019-blindness-detection/test.csv')
print(train_df.shape)
print(test_df.shape)
test_df.head()

In [ ]:
def get_pad_width(im, new_shape, is_rgb=True):
    pad_diff = new_shape - im.shape[0], new_shape - im.shape[1]
    t, b = math.floor(pad_diff[0]/2), math.ceil(pad_diff[0]/2)
    l, r = math.floor(pad_diff[1]/2), math.ceil(pad_diff[1]/2)
    if is_rgb:
        pad_width = ((t,b), (l,r), (0, 0))
    else:
        pad_width = ((t,b), (l,r))
    return pad_width

def preprocess_image(image_path, desired_size=224):
    im = Image.open(image_path)
    im = im.resize((desired_size, )*2, resample=Image.LANCZOS)
#     im = im.resize((desired_size, )*2)
    
    return im

In [ ]:
N = test_df.shape[0]
x_test = np.empty((N, 224, 224, 3), dtype=np.uint8)

for i, image_id in enumerate(tqdm(test_df['id_code'])):
    x_test[i, :, :, :] = preprocess_image(
        f'../input/aptos2019-blindness-detection/test_images/{image_id}.png'
    )

### 显示一些测试图片

我们来可视化测试集中的前10张眼球图片。快速查看后发现，只有2/10张眼球看起来正常。下面我们应该定义Ben的预处理函数。这个预处理函数会让我们更方便地看到不正常的眼球斑点，在待会儿我们使用热力图结合眼球图片来显示的时候也会更清晰。

关于Ben的预处理函数：
指的是2015年APTOS也举办过这样的比赛，有位大牛Ben，使用了这样的预处理方式：
image=cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) ,  10) ,-4 ,128)
加了高斯模糊，使得图片减少了光照的影响。但是在2019年今年的比赛中，很多人加了这样的处理后，结果变差。可能是2015年的眼球图片和今年的有所差异。但是这种方式在kernel中很火哦~

In [ ]:
# model.summary()
def load_image_ben_orig(path,resize=True,crop=False,norm255=True,keras=False):
    image = cv2.imread(path)
    
#     if crop:
#         image = crop_image(image)
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#     if resize:
#         image = cv2.resize(image,(SIZE,SIZE))
        
    image=cv2.addWeighted( image,4, cv2.GaussianBlur( image , (0,0) ,  10) ,-4 ,128)
#     image=cv2.addWeighted( image,4, cv2.medianBlur( image , 10) ,-4 ,128)
    
    # NOTE plt.imshow can accept both int (0-255) or float (0-1), but deep net requires (0-1)
    if norm255:
        return image/255
    elif keras:
        #see https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py for mode
        #see https://github.com/keras-team/keras-applications/blob/master/keras_applications/xception.py for inception,xception mode
        #the use of tf based preprocessing (- and / by 127 respectively) will results in [-1,1] so it will not visualize correctly (directly)
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

def transform_image_ben(img,resize=True,crop=False,norm255=True,keras=False):  
    image=cv2.addWeighted( img,4, cv2.GaussianBlur( img , (0,0) ,  10) ,-4 ,128)
    
    # NOTE plt.imshow can accept both int (0-255) or float (0-1), but deep net requires (0-1)
    if norm255:
        return image/255
    elif keras:
        image = np.expand_dims(image, axis=0)
        return preprocess_input(image)[0]
    else:
        return image.astype(np.int16)
    
    return image

In [ ]:
def display_samples(df, columns=5, rows=2, Ben=True):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
#         image_id = df.loc[i,'diagnosis']
        path = f'../input/aptos2019-blindness-detection/test_images/{image_path}.png'
        if Ben:
            img = load_image_ben_orig(path)
        else:
            img = cv2.imread(path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
#         plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(test_df, Ben=False)
display_samples(test_df, Ben=True)

# 3. 定义模型。获取梯度的一些技巧。

概念上，我们可以只是加载预训练模型，计算梯度，给出热力图就可以了！但是，原始模型使用了“Sequential method”来构造一个微调的DenseNet模型，而不是“Functional method”。不幸的是，Sequential method 我们不能直接获取最后一层卷积层。因此，我们不能计算激活和梯度值。所以我们需要一个技巧。

这个技巧使用了`Sequential method`，使用共享层来构造一个模型，然后应用预训练参数。然后，我用`Functional method`和同样的层构造了另一个共享所有层的模型。因为所有层都是共享的，这两个模型其实都是一样的，有一样的参数。


In [ ]:
from keras import layers
from keras.models import Model
import keras.backend as K

首先，让我们定义 `DenseNet` 的主心骨.

In [ ]:
K.clear_session()
densenet = DenseNet121(
    weights=None,
    include_top=False,
    input_shape=(None,None,3)
)

其次，我们定义和最初的kernel一样的3个共享的头部层。使用`Sequential()`模型构造（和原始模型也是一样的）。从`model.summary()`的运行结果你可以看到模型的细节你是看不到的，我们不能直接获取卷积层的最后一层。因此，我们不能从这里获取梯度。

In [ ]:
GAP_layer = layers.GlobalAveragePooling2D()
drop_layer = layers.Dropout(0.5)
dense_layer = layers.Dense(5, activation='sigmoid', name='final_output')

In [ ]:
def build_model_sequential():
    model = Sequential()
    model.add(densenet)
    model.add(GAP_layer)
    model.add(drop_layer)
    model.add(dense_layer)
    return model

In [ ]:
modelA = build_model_sequential()
modelA.load_weights('../input/aptos-data/dense_xhlulu_731.h5')

modelA.summary()

接下来，我们用同样的（共享）层构造另外一个模型。当我们预训练第一个模型参数后，第二个模型获得了同样的参数，因为所有层都是共享的。

In [ ]:
def build_model_functional():
    base_model = densenet
    
    x = GAP_layer(base_model.layers[-1].output)
    x = drop_layer(x)
    final_output = dense_layer(x)
    model = Model(base_model.layers[0].input, final_output)
    
    return model

通过使用functional模型，运行model.summary()后，我们可以获取模型的网络结构里的所有的层。因为输出的内容太长了，如果这些内容被隐藏了，你可以点击按钮展开查看更多层。

In [ ]:
model = build_model_functional() # with pretrained weights, and layers we want
model.summary()

这里，我们可以找到最后一个卷积层。注意了，我们可以使用`conv5_block16_concat`或者`relu`模块。

## 3.1 确保我们拿到了正确的参数

这个模块的目的是确保我们已经加载了正确的参数。虽然在以前的版本中，已经被证明是正确的。在这里我只是说明一下。


In [ ]:
# y_test = model.predict(x_test) > 0.5
# y_test = y_test.astype(int).sum(axis=1) - 1

# test_df['diagnosis'] = y_test
# test_df.to_csv('submission.csv',index=False)
# y_test.shape, x_test.shape

In [ ]:
# import seaborn as sns
# import cv2

# SIZE=224
# def create_pred_hist(pred_level_y,title='NoTitle'):
#     results = pd.DataFrame({'diagnosis':pred_level_y})

#     f, ax = plt.subplots(figsize=(7, 4))
#     ax = sns.countplot(x="diagnosis", data=results, palette="GnBu_d")
#     sns.despine()
#     plt.title(title)
#     plt.show()

In [ ]:
# create_pred_hist(y_test,title='predicted level distribution in test set')

# 4. 真的还是假的特征

现在我们开始最主要的模块。我们来调查一下模型的性能。首先，让我们通过calculation函数定义一个热力图。就好像介绍中说得，代码引用了[这篇文章]
(http://www.hackevolve.com/where-cnn-is-looking-grad-cam/),这篇文章引用了 F.Chollet 的书.

这个函数接收了4个参数作为输入：
(1) 待可视化的图片，记得要放入预处理过的版本。 (2) 模型 (3) 最后一层卷积层 (4) 一个用于结合热力图来展示的辅助图片; 我使用了Ben的预处理方式处理过的图片，因为这个处理方式消除了光照条件的影响。 所以特别适合最终展示。

In [ ]:
def gen_heatmap_img(img, model0, layer_name='last_conv_layer',viz_img=None,orig_img=None):
    preds_raw = model0.predict(img[np.newaxis])
    preds = preds_raw > 0.5 # use the same threshold as @xhlulu original kernel
    class_idx = (preds.astype(int).sum(axis=1) - 1)[0]
#     print(class_idx, class_idx.shape)
    class_output_tensor = model0.output[:, class_idx]
    
    viz_layer = model0.get_layer(layer_name)
    grads = K.gradients(
                        class_output_tensor ,
                        viz_layer.output
                        )[0] # gradients of viz_layer wrt output_tensor of predicted class
    
    pooled_grads=K.mean(grads,axis=(0,1,2))
    iterate=K.function([model0.input],[pooled_grads, viz_layer.output[0]])
    
    pooled_grad_value, viz_layer_out_value = iterate([img[np.newaxis]])
    
    for i in range(pooled_grad_value.shape[0]):
        viz_layer_out_value[:,:,i] *= pooled_grad_value[i]
    
    heatmap = np.mean(viz_layer_out_value, axis=-1)
    heatmap = np.maximum(heatmap,0)
    heatmap /= np.max(heatmap)

    viz_img=cv2.resize(viz_img,(img.shape[1],img.shape[0]))
    heatmap=cv2.resize(heatmap,(viz_img.shape[1],viz_img.shape[0]))
    
    heatmap_color = cv2.applyColorMap(np.uint8(heatmap*255), cv2.COLORMAP_SPRING)/255
    heated_img = heatmap_color*0.5 + viz_img*0.5
    
    print('raw output from model : ')
    print_pred(preds_raw)
    
    if orig_img is None:
        show_Nimages([img,viz_img,heatmap_color,heated_img])
    else:
        show_Nimages([orig_img,img,viz_img,heatmap_color,heated_img])
    
    plt.show()
    return heated_img

这里是展示图片和预测结果的工具函数

In [ ]:
def show_image(image,figsize=None,title=None):
    
    if figsize is not None:
        fig = plt.figure(figsize=figsize)
#     else: # crash!!
#         fig = plt.figure()
        
    if image.ndim == 2:
        plt.imshow(image,cmap='gray')
    else:
        plt.imshow(image)
        
    if title is not None:
        plt.title(title)

def show_Nimages(imgs,scale=1):

    N=len(imgs)
    fig = plt.figure(figsize=(25/scale, 16/scale))
    for i, img in enumerate(imgs):
        ax = fig.add_subplot(1, N, i + 1, xticks=[], yticks=[])
        show_image(img)
        
def print_pred(array_of_classes):
    xx = array_of_classes
    s1,s2 = xx.shape
    for i in range(s1):
        for j in range(s2):
            print('%.3f ' % xx[i,j],end='')
        print('')

首先，让我们测试前10个测试数据里的样本。对于每个数据，我展示了原始图片、经过Ben的方式处理过的图片、热力图、和热力图结合的图片。

In [ ]:
NUM_SAMP=10
SEED=77
layer_name = 'relu' #'conv5_block16_concat'
for i, (idx, row) in enumerate(test_df[:NUM_SAMP].iterrows()):
    path=f"../input/aptos2019-blindness-detection/test_images/{row['id_code']}.png"
    ben_img = load_image_ben_orig(path)
    input_img = np.empty((1,224, 224, 3), dtype=np.uint8)
    input_img[0,:,:,:] = preprocess_image(path)
        
    print('test pic no.%d' % (i+1))
    _ = gen_heatmap_img(input_img[0],
                        model, layer_name=layer_name,viz_img=ben_img)

我们得到了很多有趣的见解。

* 第1、4、5、6预测结果看起来不错
* 第2个预测没有指出中间大大的斑点
* 第3和第7预测结果同样错过了重要的斑点
* 第9张图片，到处都是血斑点，模型只指出4处，并没有指出全部的斑点
* 第8、10图片看起来比较正常，但是模型似乎获取了伪特征，将其定义为严重程度为1。


# 5. 使用Albumentation进行鲁棒性测试

在这个模块，为了看模型是否给出正确的预测，我会向大家展示5个albumentation的图片变换方式，然后使用我们的模型进行测试。第6和最终的图片增强是将这5种变换结合起来！你可以看到这些样本在下面一行行排列。注意了，第一张图片是原始图片。

In [ ]:
from albumentations import *
import time

IMG_SIZE = (224,224)

'''Use case from https://www.kaggle.com/alexanderliao/image-augmentation-demo-with-albumentation/'''
def albaugment(aug0, img):
    return aug0(image=img)['image']
idx=8
image1=x_test[idx]

'''1. Rotate or Flip'''
aug1 = OneOf([
    Rotate(p=0.99, limit=160, border_mode=0,value=0), # value=black
    Flip(p=0.5)
    ],p=1)

'''2. Adjust Brightness or Contrast'''
aug2 = RandomBrightnessContrast(brightness_limit=0.45, contrast_limit=0.45,p=1)
h_min=np.round(IMG_SIZE[1]*0.72).astype(int)
h_max= np.round(IMG_SIZE[1]*0.9).astype(int)
print(h_min,h_max)

'''3. Random Crop and then Resize'''
#w2h_ratio = aspect ratio of cropping
aug3 = RandomSizedCrop((h_min, h_max),IMG_SIZE[1],IMG_SIZE[0], w2h_ratio=IMG_SIZE[0]/IMG_SIZE[1],p=1)

'''4. CutOut Augmentation'''
max_hole_size = int(IMG_SIZE[1]/10)
aug4 = Cutout(p=1,max_h_size=max_hole_size,max_w_size=max_hole_size,num_holes=8 )#default num_holes=8

'''5. SunFlare Augmentation'''
aug5 = RandomSunFlare(src_radius=max_hole_size,
                      num_flare_circles_lower=10,
                      num_flare_circles_upper=20,
                      p=1)#default flare_roi=(0,0,1,0.5),

'''6. Ultimate Augmentation -- combine everything'''
final_aug = Compose([
    aug1,aug2,aug3,aug4,aug5
],p=1)


img1 = albaugment(aug1,image1)
img2 = albaugment(aug1,image1)
print('Rotate or Flip')
show_Nimages([image1,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug2,image1)
img2 = albaugment(aug2,image1)
img3 = albaugment(aug2,image1)
print('Brightness or Contrast')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug3,image1)
img2 = albaugment(aug3,image1)
img3 = albaugment(aug3,image1)
print('Rotate and Resize')
show_Nimages([img3,img1,img2],scale=2)
print(img1.shape,img2.shape)
# time.sleep(1)

img1 = albaugment(aug4,image1)
img2 = albaugment(aug4,image1)
img3 = albaugment(aug4,image1)
print('CutOut')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(aug5,image1)
img2 = albaugment(aug5,image1)
img3 = albaugment(aug5,image1)
print('Sun Flare')
show_Nimages([img3,img1,img2],scale=2)
# time.sleep(1)

img1 = albaugment(final_aug,image1)
img2 = albaugment(final_aug,image1)
img3 = albaugment(final_aug,image1)
print('All above combined')
show_Nimages([img3,img1,img2],scale=2)
print(img1.shape,img2.shape)


现在让我们一起看我们的模型是怎样识别不同的图片增强的。注意了，这个测试图片模型预测了没有图片增强，并预测为严重程度为3 (score `[0.998 1.000 0.999 0.953 0.068]`)

因为这个增强是随机的，你可以看到不同的结果。在我的实验中，这个模型是相当鲁棒的，因为它基本上预测出一样的严重程度，除了将所有增强结合起来的那个有时候会预测为等级4，检测到的特征也都是比较正确的。

主要是你可以使用所有的直觉来调整你的图片增强的架构，使得模型更加鲁棒。

我们本来可以测试更多的图片，但是我会将剩下的都留给你去做。

In [ ]:
aug_list = [aug5, aug2, aug3, aug4, aug1, final_aug]
aug_name = ['SunFlare', 'brightness or contrast', 'crop and resized', 'CutOut', 'rotate or flip', 'Everything Combined']

idx=8
layer_name = 'relu' #'conv5_block16_concat'
for i in range(len(aug_list)):
    path=f"../input/aptos2019-blindness-detection/test_images/{test_df.iloc[idx]['id_code']}.png"
    input_img = np.empty((1,224, 224, 3), dtype=np.uint8)
    input_img[0,:,:,:] = preprocess_image(path)
    aug_img = albaugment(aug_list[i],input_img[0,:,:,:])
    ben_img = transform_image_ben(aug_img)
    
    print('test pic no.%d -- augmentation: %s' % (i+1, aug_name[i]))
    _ = gen_heatmap_img(aug_img,
                        model, layer_name=layer_name,viz_img=ben_img,orig_img=input_img[0])

热力图可视化有很多新颖的应用：

* 可视化增强后的图片，来看你的模型是否足够鲁棒（预测结果一样），或者模型是否真的理解了图片（是否有保留重要信息）

* 如果模型存在过拟合，可视化训练集来看一些伪特征。设计一些高效的图片增强方式来消除伪特征。

* 可视化0~4严重级别的图片，去找决定每个严重等级的特征。


结束了，希望这个notebook可以帮助到你哦~

-- 